In [26]:
import requests
import os

BASE_URL = "https://openapi.data.uwaterloo.ca/v3"
API_KEY = os.getenv("UW_TOKEN")
HEADERS = {
    "accept": "application/json",
    "x-api-key": API_KEY
}

def register_account(email, project: str = "Flow", uri: str = "https://github.com/AndreyKarmanov/Flow") -> requests.Response:
    url = BASE_URL + "/account/register"
    headers = {
        "content-type": "application/x-www-form-urlencoded"
    }

    data = {
        "email": email,
        "project": project,
        "uri": uri
    }

    # and then just use the curl command in CMD (not powershell) to get activated. The email has the api token. 
    return requests.post(url, headers=headers, data=data)

In [47]:
def get_departments():
    """
    Returns a list of all departments, and their `organizationCodes`
    """
    return requests.get(url=f'{BASE_URL}/AcademicOrganizations',
                        headers=HEADERS).json()

def get_subjects(organizationCode=None):
    """
    `organizationCode`: filters subjects by organization code
    """
    if not organizationCode:
        return requests.get(url=f'{BASE_URL}/Subjects',
                            headers=HEADERS).json()

    return requests.get(url=f'{BASE_URL}/Subjects/associatedto/{organizationCode}',
                        headers=HEADERS).json()


def get_current_term():
    return requests.get(url=f'{BASE_URL}/Terms/current',
                        headers=HEADERS).json()

def get_terms(year=None):
    """
    `year`: optionally filters terms by year
    """
    if year:
        return requests.get(url=f'{BASE_URL}/Terms/foracademicyear/{year}',
                            headers=HEADERS).json()
    return requests.get(url=f'{BASE_URL}/Terms', 
                        headers=HEADERS).json()


def get_current_term():
    return requests.get(url=f'{BASE_URL}/Terms/current',
                        headers=HEADERS).json()


def get_courses(termCode, subject=None, courseId=None):

    if courseId:
        return requests.get(url=f'{BASE_URL}/Courses/{termCode}/{courseId}',
                            headers=HEADERS).json()

    elif subject:
        return requests.get(url=f'{BASE_URL}/Courses/{termCode}/{subject}',
                            headers=HEADERS).json()

    return requests.get(url=f'{BASE_URL}/Courses/{termCode}',
                        headers=HEADERS).json()

In [44]:
currentTerm = get_current_term()

departments = get_departments()

subjects = get_subjects(departments[0]['code'])

In [49]:
termCode = currentTerm['termCode']
accDepartment = departments[0]['code']

In [50]:
courses = get_courses(termCode)

In [57]:
# get all terms, and then get all courses for each term.
# combine all the courses into one list, taking the newest version of the course if there are duplicates.

import json

data = [{ 'term' : term, 'courses' : get_courses(term['termCode']) } for term in get_terms()]

with open('data.json', 'w') as outfile:
    json.dump(data, outfile)

In [1]:
import json

data = json.loads(open('data.json').read())

In [12]:
data[1]['courses'][0]

{'courseId': '000003',
 'courseOfferNumber': 1,
 'termCode': '1209',
 'termName': 'Fall 2020',
 'associatedAcademicCareer': 'GRD',
 'associatedAcademicGroupCode': 'ART',
 'associatedAcademicOrgCode': 'ACC',
 'subjectCode': 'ACC',
 'catalogNumber': '610',
 'title': 'Public Accounting Practice',
 'descriptionAbbreviated': 'Public Accounting Practice',
 'description': "This course is designed to enhance the students' abilities to apply knowledge of accounting principles, theories, and practices through the use of multi-dimensional case studies.",
 'gradingBasis': 'NUM',
 'courseComponentCode': 'LEC',
 'enrollConsentCode': 'D',
 'enrollConsentDescription': 'No Consent Required',
 'dropConsentCode': 'N',
 'dropConsentDescription': 'No Consent Required',
 'requirementsDescription': None}

In [16]:
# going from new to old terms, add courses to the list if they don't already exist
courseSet = set()
courses = []

sorted_data = sorted(data, key=lambda x: x['term']['termCode'], reverse=True)
# filter out terms that had errors

for term in sorted(data, key=lambda x: x['term']['termCode'], reverse=True):
    for course in term['courses']:
        try:
            code = course['subject'] + course['subjectCode']
        except:
            print(term['courses'])
            raise
        if code not in courseSet:
            courseSet.add(code)
            courses.append(course)

{'type': 'https://tools.ietf.org/html/rfc7231#section-6.5.4', 'title': 'Not Found', 'status': 404, 'traceId': '00-9d397f8185d56487950009b69489bd82-5a7e4c2eac857243-00'}


TypeError: string indices must be integers, not 'str'